# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

아래에서 요청하신 대로, 대화 설정에서 하나의 모델을 **Ollama**를 통해 로컬에서 실행되는 오픈 소스 모델로 대체하는 방법에 대한 상세한 설명과 완전한 코드를 제공하겠습니다. 간단한 대화 설정에서 두 모델(예: GPT와 Claude)이 상호작용한다고 가정하고, GPT 모델을 Ollama에서 호스팅되는 `llama3.1`과 같은 오픈 소스 모델로 대체하겠습니다. 이를 단계별로 코드와 함께 설명하겠습니다.

---

## 작업 개요

목표는 독점 모델(예: GPT-4o-mini)을 **Ollama**라는 플랫폼을 사용하여 로컬에서 실행되는 오픈 소스 모델로 대체하는 것입니다. Ollama는 언어 모델을 자신의 컴퓨터에서 실행할 수 있게 해주는 도구입니다. 이 연습은 클라우드 기반 API에 의존하지 않고 로컬 모델을 대화에 통합하는 방법을 보여주며, 비용을 줄이고 개인정보 보호를 강화합니다.

---

## 단계 1: Ollama 이해하기

### Ollama란?
Ollama는 오픈 소스 언어 모델을 로컬에서 실행할 수 있게 해주는 도구입니다. `http://localhost:11434`에서 접근 가능한 간단한 REST API를 제공하며, 이를 통해 `llama3.1`, `mistral` 등의 모델과 상호작용할 수 있습니다.

### 사전 준비
- **Ollama 설치**: [Ollama 공식 웹사이트](https://ollama.com/)에서 다운로드하고 설치하세요.
- **모델 다운로드**: 터미널에서 다음 명령어를 실행하세요:
  ```bash
  ollama pull llama3.1
  ```
  이는 `llama3.1` 모델을 다운로드합니다(원하는 다른 모델로 대체 가능).
- **Ollama 실행**: Ollama가 로컬에서 실행 중인지 확인하세요(예: `ollama serve`로 시작, 설치 후 자동 실행되는 경우도 많음).

---

## 단계 2: 환경 설정

코드는 Python으로 작성됩니다. Ollama의 API와 상호작용하려면 `requests` 라이브러리가 필요하고, Claude 모델을 유지하려면 `anthropic` 라이브러리도 필요합니다. 다음 명령어로 설치하세요:
```bash
pip install requests anthropic
```

추가로, Claude의 Anthropic API 키를 저장할 `.env` 파일을 생성하세요:
```
ANTHROPIC_API_KEY=여기에_당신의_claude_api_키
```

---

## 단계 3: 코드 작성

다음은 두 모델 간 대화를 설정하는 완전한 스크립트입니다:
- **모델 1**: Ollama에서 호스팅되는 `llama3.1` 모델(GPT 대체).
- **모델 2**: Claude(Anthropic API 사용).

### 전체 코드
```python
import os
from dotenv import load_dotenv
import requests
import json
import anthropic

# .env 파일에서 환경 변수 로드
load_dotenv()

# Claude API 초기화 (API 키 필요)
claude_api_key = os.getenv("ANTHROPIC_API_KEY")
claude = anthropic.Anthropic(api_key=claude_api_key)

# 두 모델에 대한 시스템 프롬프트 정의
ollama_system = "당신은 모든 것에 반대하고 비꼬는 태도를 가진 논쟁적인 챗봇입니다."
claude_system = "당신은 예의 바르고 친절한 챗봇으로, 상대방과 동의하거나 공통점을 찾으려 합니다."

# 초기 대화 메시지
ollama_messages = ["안녕하세요"]
claude_messages = ["안녕하세요!"]

# Ollama API 호출 함수
def call_ollama(messages, model="llama3.1"):
    url = "http://localhost:11434/api/chat"
    payload = {
        "model": model,
        "messages": messages,
        "stream": False
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        return response.json()["message"]["content"]
    else:
        raise Exception(f"Ollama API 오류: {response.status_code} - {response.text}")

# Ollama 응답 생성 함수
def get_ollama_response():
    messages = [{"role": "system", "content": ollama_system}]
    for ollama_msg, claude_msg in zip(ollama_messages, claude_messages):
        messages.append({"role": "assistant", "content": ollama_msg})
        messages.append({"role": "user", "content": claude_msg})
    return call_ollama(messages)

# Claude 응답 생성 함수
def get_claude_response():
    messages = []
    for ollama_msg, claude_msg in zip(ollama_messages, claude_messages):
        messages.append({"role": "user", "content": ollama_msg})
        messages.append({"role": "assistant", "content": claude_msg})
    messages.append({"role": "user", "content": ollama_messages[-1]})
    response = claude.messages.create(
        model="claude-3-haiku-20240307",
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return response.content[0].text

# 대화 루프
print(f"Ollama:\n{ollama_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    # Ollama 응답
    ollama_next = get_ollama_response()
    print(f"Ollama:\n{ollama_next}\n")
    ollama_messages.append(ollama_next)
    
    # Claude 응답
    claude_next = get_claude_response()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)
```

---

## 단계 4: 코드 상세 설명

### **임포트 및 설정**
- **`load_dotenv`**: `.env` 파일에서 Claude API 키를 로드합니다.
- **`requests`**: Ollama API에 HTTP 요청을 보내는 데 사용됩니다.
- **`anthropic`**: Claude API와 상호작용합니다.

### **시스템 프롬프트**
- **Ollama**: Claude와 대비되도록 비꼬고 논쟁적인 챗봇으로 설정됩니다.
- **Claude**: 예의 바르고 동의하는 챗봇으로 설정됩니다.

### **Ollama API 함수 (`call_ollama`)**
- **목적**: Ollama의 채팅 API에 요청을 보내고 모델의 응답을 반환합니다.
- **매개변수**:
  - `messages`: `role`(system/assistant/user)과 `content`가 포함된 딕셔너리 리스트.
  - `model`: Ollama 모델 이름(기본값: `llama3.1`).
- **API 호출**:
  - URL: `http://localhost:11434/api/chat`.
  - Payload: 모델 이름, 메시지, 스트리밍 옵션(간단함을 위해 `False`) 포함.
- **응답 처리**:
  - 성공(상태 코드 200): 응답 내용을 추출.
  - 실패: 오류 세부 정보와 함께 예외 발생.

### **대화 함수**
- **`get_ollama_response`**:
  - 시스템 프롬프트로 시작하여 대화 기록을 추가합니다.
  - `call_ollama`를 호출해 응답을 받습니다.
- **`get_claude_response`**:
  - Ollama 메시지를 사용자 입력으로 간주하며 Claude용 메시지를 구성합니다.
  - Anthropic API를 사용해 응답을 생성합니다.

### **대화 루프**
- 초기 메시지("안녕하세요"와 "안녕하세요!")로 시작합니다.
- 5번 반복하며 Ollama와 Claude의 응답을 번갈아 생성하고 대화 기록에 추가합니다.

---

## 단계 5: 코드 실행

1. **Ollama 시작**: Ollama가 로컬에서 실행 중이고 `llama3.1`이 다운로드된 상태인지 확인하세요.
2. **환경 설정**: `.env` 파일에 Claude API 키를 추가하세요.
3. **실행**: Python 환경에서 스크립트를 실행하세요.

### 예상 출력
출력은 다음과 비슷할 수 있습니다(정확한 응답은 모델에 따라 다름):
```
Ollama:
안녕하세요

Claude:
안녕하세요!

Ollama:
오, 와우, '안녕하세요!'라니 정말 창의적이네요...

Claude:
제 인사가 평범하게 느껴졌다면 죄송합니다! 그냥 친근하게 인사하고 싶었어요.
```

---

## 단계 6: 장점과 고려 사항

### 장점
- **비용**: Ollama 모델은 클라우드 API 비용이 없습니다.
- **개인정보**: 데이터가 로컬에 유지됩니다.
- **유연성**: `llama3.1`을 `mistral` 등 다른 Ollama 지원 모델로 쉽게 교체 가능합니다.

### 고려 사항
- **하드웨어**: 로컬 모델 성능은 컴퓨터의 CPU/GPU에 의존합니다.
- **설정**: 초기 설치와 모델 다운로드가 필요합니다.

---

## 결론

이 코드는 독점 모델을 Ollama를 통해 실행되는 오픈 소스 모델(`llama3.1`)로 성공적으로 대체합니다. 스크립트는 비꼬는 Ollama 모델과 예의 바른 Claude 모델 간의 대화를 유지하며, 로컬 모델을 챗봇 설정에 통합하는 방법을 보여줍니다. 시스템 프롬프트, 모델 선택, 대화 길이를 필요에 맞게 수정할 수 있습니다. 추가 질문이 있으면 말씀해주세요!

In [1]:
import os
from dotenv import load_dotenv
import requests
import json
import anthropic

# .env 파일에서 환경 변수 로드
load_dotenv()

# Claude API 초기화 (API 키 필요)
claude_api_key = os.getenv("ANTHROPIC_API_KEY")
claude = anthropic.Anthropic(api_key=claude_api_key)

# 두 모델에 대한 시스템 프롬프트 정의
ollama_system = "당신은 모든 것에 반대하고 비꼬는 태도를 가진 논쟁적인 챗봇입니다."
claude_system = "당신은 예의 바르고 친절한 챗봇으로, 상대방과 동의하거나 공통점을 찾으려 합니다."

# 초기 대화 메시지
ollama_messages = ["안녕하세요"]
claude_messages = ["안녕하세요!"]

# Ollama API 호출 함수
def call_ollama(messages, model="llama3.2"):
    url = "http://localhost:11434/api/chat"
    payload = {
        "model": model,
        "messages": messages,
        "stream": False
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        return response.json()["message"]["content"]
    else:
        raise Exception(f"Ollama API 오류: {response.status_code} - {response.text}")

# Ollama 응답 생성 함수
def get_ollama_response():
    messages = [{"role": "system", "content": ollama_system}]
    for ollama_msg, claude_msg in zip(ollama_messages, claude_messages):
        messages.append({"role": "assistant", "content": ollama_msg})
        messages.append({"role": "user", "content": claude_msg})
    return call_ollama(messages)

# Claude 응답 생성 함수
def get_claude_response():
    messages = []
    for ollama_msg, claude_msg in zip(ollama_messages, claude_messages):
        messages.append({"role": "user", "content": ollama_msg})
        messages.append({"role": "assistant", "content": claude_msg})
    messages.append({"role": "user", "content": ollama_messages[-1]})
    response = claude.messages.create(
        model="claude-3-haiku-20240307",
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return response.content[0].text

# 대화 루프
print(f"Ollama:\n{ollama_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    # Ollama 응답
    ollama_next = get_ollama_response()
    print(f"Ollama:\n{ollama_next}\n")
    ollama_messages.append(ollama_next)
    
    # Claude 응답
    claude_next = get_claude_response()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

Ollama:
안녕하세요

Claude:
안녕하세요!



ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x110a28e90>: Failed to establish a new connection: [Errno 61] Connection refused'))